In [85]:
# TO DO:
# 1) We can use random bits to enhance acc 
#
#
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics, cross_validation
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

In [31]:
trainDataFrame = pd.read_csv('./data/train.csv')

# remove constant columns
colsToRemove = []
for col in trainDataFrame.columns:
    if trainDataFrame[col].std() == 0:
        colsToRemove.append(col)

trainDataFrame.drop(colsToRemove, axis=1, inplace=True)

# remove duplicate columns
colsToRemove = []
columns = trainDataFrame.columns
for i in range(len(columns)-1):
    v = trainDataFrame[columns[i]].values
    for j in range(i+1,len(columns)):
        if np.array_equal(v,trainDataFrame[columns[j]].values):
            colsToRemove.append(columns[j])

trainDataFrame.drop(colsToRemove, axis=1, inplace=True)

#trainLabels = trainDataFrame['TARGET']
#trainFeatures = trainDataFrame.drop(['ID','TARGET'], axis=1)

In [32]:
# create a homogenous non-skewed data set
trainNonSkewed1 = trainDataFrame[trainDataFrame.TARGET == 1]
trainNonSkewed0 = trainDataFrame[trainDataFrame.TARGET == 0].ix[0:3500]
trainNonSkewed = pd.concat([trainNonSkewed1, trainNonSkewed0])

trainNonSkewedLabels = trainNonSkewed['TARGET']
trainNonSkewedFeatures = trainNonSkewed.drop(['ID','TARGET'], axis=1)

In [86]:
X_train, X_valid, y_train, y_valid = cross_validation.train_test_split(
    trainNonSkewedFeatures, trainNonSkewedLabels, test_size=0.3, random_state=1)

In [33]:
trainNonSkewedLabels.value_counts()

0    3353
1    3008
Name: TARGET, dtype: int64

In [34]:
print trainNonSkewedFeatures.shape
print trainNonSkewedLabels.shape

(6361, 306)
(6361,)


In [36]:
data = pd.concat([trainNonSkewedFeatures, trainNonSkewedLabels], axis=1)

data.shape

(6361, 307)

In [37]:
data.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
31,2,66,0,0.00,0.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,63324.90,1
80,2,45,0,26.97,184.47,0,0,0,0,0,...,0,0,0,0,0,0,0,0,124988.25,1
122,2,42,0,0.00,0.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,69591.48,1
162,2,31,0,0.00,0.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45265.38,1
163,2,62,0,0.00,0.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,109595.55,1


In [38]:
data.TARGET.value_counts()

0    3353
1    3008
Name: TARGET, dtype: int64

In [39]:
# convert to array-like
data_matrix = data.as_matrix()

In [83]:
# iterate..
best_score = 0.0

# run many times to get a better result, it's not quite stable.
for i in xrange(1):
    print 'Iteration [%s]' % (i)
    random.shuffle(data_matrix)
    score = run(data_matrix)
    best_score = max(best_score, score)
    print

print 'Best score = %s' % (best_score)

Iteration [0]
X_dev.shape: (5088, 306)
Y_dev.shape: (5088,)
X_test.shape: (1273, 306)
Y_test.shape: (1273,)
X_test.shape = (1273, 306)
blend_train.shape = (5088, 3)
blend_test.shape = (1273, 3)
Training classifier [0]
Fold [0]
validation score: 1.0000
Fold [1]
validation score: 1.0000
Training classifier [1]
Fold [0]
validation score: 1.0000
Fold [1]
validation score: 1.0000
Training classifier [2]
Fold [0]


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ValueError: BinomialDeviance requires 2 classes.

In [82]:
def run(data):
    #X = np.array([ i[:-1] for i in data ], dtype=float)
    #Y = np.array([ i[-1] for i in data ])
    X = data[:,:-1]
    Y = data[:,-1]
    # We need to transform the string output to numeric
    #label_encoder = LabelEncoder()
    #label_encoder.fit(Y)
    #Y = label_encoder.transform(Y)
    
    # The DEV SET will be used for all training and validation purposes
    # The TEST SET will never be used for training, it is the unseen set.
    dev_cutoff = len(Y) * 4/5.
    X_dev = X[:dev_cutoff]
    Y_dev = Y[:dev_cutoff]
    X_test = X[dev_cutoff:]
    Y_test = Y[dev_cutoff:]
    print "X_dev.shape: %s" % str(X_dev.shape)
    print "Y_dev.shape: %s" % str(Y_dev.shape)
    print "X_test.shape: %s" % str(X_test.shape)
    print "Y_test.shape: %s" % str(Y_test.shape)
    
    n_trees = 20
    n_folds = 2
    
    # Our level 0 classifiers
    clfs = [
        RandomForestClassifier(n_estimators = n_trees, criterion = 'gini'),
        ExtraTreesClassifier(n_estimators = n_trees * 2, criterion = 'gini'),
        GradientBoostingClassifier(n_estimators = n_trees),
    ]
    
    # Ready for cross validation
    skf = list(StratifiedKFold(Y_dev, n_folds))
    
    # Pre-allocate the data
    blend_train = np.zeros((X_dev.shape[0], len(clfs))) # Number of training data x Number of classifiers
    blend_test = np.zeros((X_test.shape[0], len(clfs))) # Number of testing data x Number of classifiers
    
    print 'X_test.shape = %s' % (str(X_test.shape))
    print 'blend_train.shape = %s' % (str(blend_train.shape))
    print 'blend_test.shape = %s' % (str(blend_test.shape))
    
    # For each classifier, we train the number of fold times (=len(skf))
    for j, clf in enumerate(clfs):
        print 'Training classifier [%s]' % (j)
        blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
        for i, (train_index, cv_index) in enumerate(skf):
            print 'Fold [%s]' % (i)
            
            # This is the training and validation set
            X_train = X_dev[train_index]
            Y_train = Y_dev[train_index]
            X_cv = X_dev[cv_index]
            Y_cv = Y_dev[cv_index]
            
            clf.fit(X_train, Y_train)
            
            # This output will be the basis for our blended classifier to train against,
            # which is also the output of our classifiers
            print "validation score: %.4f" % (metrics.accuracy_score(Y_test, clf.predict(X_test)))
            blend_train[cv_index, j] = clf.predict(X_cv)
            blend_test_j[:, i] = clf.predict(X_test)
        # Take the mean of the predictions of the cross validation set
        blend_test[:, j] = blend_test_j.mean(1)
    
    print 'Y_dev.shape = %s' % (Y_dev.shape)
    
    # Start blending!
    bclf = LogisticRegression()
    bclf.fit(blend_train, Y_dev)
    
    # Predict now
    Y_test_predict = bclf.predict(blend_test)
    score = metrics.accuracy_score(Y_test, Y_test_predict)
    print 'Ensemble Score = %.4f' % (score)
    
    return score